In [1]:
from datasets import list_datasets

# display all datasets
all_datasets = list_datasets()
print(f"There are {len(all_datasets)} datasets currently available on the Hub")
print(f"The first 10 are: {all_datasets[:10]}")

In [2]:
from datasets import load_dataset

emotions = load_dataset("emotion")
emotions

In [3]:
train_ds = emotions["train"]
train_ds # contains a Dataset class

In [4]:
len(train_ds)

In [5]:
train_ds[0]

In [6]:
train_ds.column_names

In [7]:
# can see the data types under the 
# value is a string
# label is  ClassLabel that contains inforamtion about class names and mapping to integers
print(train_ds.features)

In [8]:
# can access several rows via a slice
train_ds[:5]

In [9]:
# or just full columns by name
train_ds["text"][:5]

Sidebar: Can also load in datasets from csv,txt and json files with `load_dataset` function, and pass `data_files` argument specifying the path/URL to one/more files. Can also specify the delimiter with `sep` and column names. Datasets documentation gives a more complete overview.

In [10]:
import pandas as pd

# convenient to convert to PD dataframe so we can access APIs for data viz
# Datasets provides a set_format() that allows us to change the output format of the dataset
emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()